# Multilevel Monte Carlo with Surrogate Models for Resource Adequacy Assessment
Author: Ensieh Sharifnia and Simon Tindemans, Delft University of Technology, 2021-2022.
Released under the MIT license.

This notebook generates results that are similar\* to those in the paper *Multilevel Monte Carlo with Surrogate Models for
Resource Adequacy Assessment* by Ensieh Sharifnia and Simon Tindemans, to be presented at the PMAPS 2022 conference. 

A preprint of the paper is available here: https://arxiv.org/abs/2203.03437. If you use (portions of) this code, please reference the published paper.

\*: Similarity only due to dependence on execution time

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import MCCoordinator
import StorageSampler

In [ ]:
import logging
# initialise logging facility
log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
    # initialise global logging facility

logging.basicConfig(level=logging.INFO)



# Storage case study

In [ ]:
def storage_system(train_size, **kwargs):
    """
    Instantiate StorageSystem object using GB system data

    :param wind_power: assumed wind power capacity (in MW)
    :param kwargs: additional arguments to be supplied to the StorageSystem constructor
    :return: StorageSystem object
    """
    import pandas as pd
    import os
    data = pd.read_csv('../data/UKdata/20161213_uk_wind_solar_demand_temperature.csv',
                       parse_dates=['UTC Time', 'Local Time'], infer_datetime_format=True, dayfirst=True, index_col=0)

    demand_data = data['demand_net'].dropna()['2006':'2015']
    wind_data = 10000 * data['wind_merra1'].dropna()

    demand_samples = {yeardata[0]: yeardata[1].values[:8760] for yeardata in
                      demand_data.groupby(demand_data.index.year)}
    wind_samples = {yeardata[0]: yeardata[1].values[:8760] for yeardata in wind_data.groupby(wind_data.index.year)}

    dataframe = pd.read_csv('../data/UKdata/battery_data.csv')
    store_power_list=3*dataframe['Power (MW)'][0:27]
    store_energy_list=3*dataframe['Energy (MWh)'][0:27]

    return StorageSampler.StorageSystem(demand_samples=demand_samples, 
                                        wind_samples=wind_samples, 
                                        store_power_list=store_power_list,
                                        store_energy_list=store_energy_list,
                                         train_size=train_size,
                                        **kwargs)

In [ ]:
def run_MLMC(system, samples, n_run, time_seconds, train_size, ml_hierarchy, use_joblib, verbose):
    '''
    Run MLMC simulation
    Parameters:
        samples: int
            initial number of samples in each level for sigma estimation.
        n_run: int
            number of runs for MLMC simulation.
        time_seconds: int
            duration for each run of MLMC simulation.
        train_size: int
            number of training samples for training surrogate models
        ml_hierarchy: {'OptimalNStore', 'GreedyNStore', 'AIGreedyNStore', 'Greedy1Store', 'AvgStore', 'NoStore', 'AIModel'}
            set of models for MLMC structure.
        use_joblib: bool
            if Ture: use all cores, otherwise run on a single core.
        Verbose: bool
            if Ture: print with details, otherwise: print summery of results
    '''
    mcc = MCCoordinator.MCCoordinator(factory=system, 
                                    ml_hierarchy=ml_hierarchy, 
                                    use_expectations=True, 
                                    use_joblib=use_joblib, joblib_n_jobs=-1, joblib_batch_size=5)
    mcc.explore(n_samples=samples)
    for i in range(n_run):
        mcc.run_recommended(time_seconds=time_seconds, verbose=verbose, optimization_target='EENS')
    mcc.verbose_result()

## Run MLMC for different structures
used in Table I & Fig 1 & Fig 2

In [ ]:
# initialize parameters for run
samples = 500
n_run = 50
time_seconds = 500
train_size = 5000
use_joblib = False
verbose = False
system = storage_system(train_size=train_size)

In [ ]:
# Estimator: MC , Architecture: Exact
ml_hierarchy = ['OptimalNStore']
run_MLMC(system, samples, n_run, time_seconds, train_size, ml_hierarchy, use_joblib, verbose)

In [ ]:
# Estimator: TS20 base , Architecture: Exact|Avg
ml_hierarchy = ['OptimalNStore', 'AvgStore']
run_MLMC(system, samples, n_run, time_seconds, train_size, ml_hierarchy, use_joblib, verbose)

In [ ]:
# Estimator: Surr , Architecture: Exact|HGB+SVR
ml_hierarchy = ['OptimalNStore', 'AIModel']
run_MLMC(system, samples, n_run, time_seconds, train_size, ml_hierarchy, use_joblib, verbose)

In [ ]:
# Estimator: TS20 full , Architecture: Exact | Gre | Avg
ml_hierarchy = ml_hierarchy = ['OptimalNStore', 'GreedyNStore', 'AvgStore']
run_MLMC(samples, n_run, time_seconds, train_size, ml_hierarchy, use_joblib, verbose)

In [ ]:
# Estimator: Surr+base , Architecture: Exact | HGB+SVR | Avg
ml_hierarchy = ['OptimalNStore', 'AIModel', 'AvgStore']
run_MLMC(system, samples, n_run, time_seconds, train_size, ml_hierarchy, use_joblib, verbose)

In [ ]:
# Estimator:Hybrid+base, Architecture:  Exact | HGB+Gre | Avg
ml_hierarchy = ['OptimalNStore', 'AIGreedyNStore', 'AvgStore']
run_MLMC(system, samples, n_run, time_seconds, train_size, ml_hierarchy, use_joblib, verbose)

In [ ]:
# Estimator:Full, Architecture:  Exact | HGB+Gre | HGB+SVR | Avg
ml_hierarchy = ['OptimalNStore', 'AIGreedyNStore', 'AIModel', 'AvgStore']
run_MLMC(system, samples, n_run, time_seconds, train_size, ml_hierarchy, use_joblib, verbose)

## TABLE II
Effects of training sample size on surrogate accuracy

In [ ]:
def scientific_notation(risk_value, std_error):
        '''
        providing scientific notation for risk value along with standard deviation
        Parameters:
            risk_value: float
            std_error: float
        Returns:
            string of scientific notation
        '''
        EFFECTIVE_ZERO = 1e-7
        if std_error < EFFECTIVE_ZERO and risk_value< EFFECTIVE_ZERO:
            return '{}({})'.format(int(risk_value), int(std_error))
        std_error_e = "{:.5e}".format(std_error)
        risk_e = "{:.10e}".format(risk_value)
        std_error_power = int(std_error_e[std_error_e.find('e')+1:])
        risk_power = int(risk_e[risk_e.find('e')+1:])
        precision = risk_power - std_error_power
        
        std_e = int(np.round(float(std_error_e[0:std_error_e.find('e')])))
        if (std_e < 3):
            precision +=1
            std_error_e = "{:.2e}".format(std_error)
            std_e = int(np.round(float(std_error_e[0:std_error_e.find('e')])*10))
        f = "{:."+str(precision)+"e}"
        risk_e = f.format(risk_value)
        
        if risk_power>3 or risk_power<-3 :
            return '{}({}).10^{}'.format(risk_e[0:risk_e.find('e')], std_e, risk_power)
        elif risk_power<4 and std_error_power>=0:
            if std_error_power==0 and int(np.round(std_error))<3 :
                f = "{:.1e}"
                risk_e = float(risk_e)
            else:
                std_e = int(np.round(std_error))
                risk_e = int(np.round(risk_value))
            return "{}({})".format(risk_e, std_e)
        else:
            return "{}({})".format(float(risk_e), std_e)

In [ ]:
# initialize parameters for run
train_size = [500, 1000, 5000]
n_run = 5
from sklearn.metrics import  mean_squared_error
import MachineLearning
print("_______________________________________________________________________")
print ("Surrogate model      Train size         Average RMSE      RMSE unit")
print("_______________________________________________________________________")
lol_arr = np.zeros(n_run)
ens_arr = np.zeros(n_run)
ML = MachineLearning.MachineLearning(train_size= 5, use_real_lol=True)
X_test = ML.load_data("../data/AIdata/daily_margin_test.csv")
ens_test = ML.load_data("../data/AIdata/ens_test.csv")
lol_test = ML.load_data("../data/AIdata/lol_test.csv")
for st in train_size:
    for i in range(n_run):
        ML = MachineLearning.MachineLearning(train_size= st, use_real_lol=True)
        X_test = ML.load_data("../data/AIdata/daily_margin_test.csv")
        lol_hat, ens_hat = ML.predict(X_test)
        lol_arr[i] = mean_squared_error( lol_test, lol_hat, squared=False)
        ens_arr[i] = mean_squared_error( ens_test, ens_hat, squared=False)

    lole = np.mean(lol_arr)
    lol_stdr = np.std(lol_arr, axis=0, ddof = 1)/ np.sqrt(lol_arr.shape[0])
    print(f"HGBRT                  {st}                {scientific_notation(lole, lol_stdr)}        (h/y)")
    eens = np.mean(ens_arr)
    eens_stdr = np.std(ens_arr, axis=0, ddof = 1)/ np.sqrt(ens_arr.shape[0])
    print(f"SVR                    {st}                {scientific_notation(eens, eens_stdr)}           (MWh/y)")


## TABLE III
Effect of surrogate model accuracy on MLMC performance
NOTE: for TS20 full, and training size = 5000, we used table I data.
Therefore, we just need to compute Hybrid+base and Full estimator with training sample =[500, 1000]

In [ ]:
# initialize parameters for run
samples = 500
n_run = 50
time_seconds = 500
train_size = 5000
use_joblib = False
verbose = False
train_size = [500, 1000]
for st in train_size: 
    system = storage_system(train_size=st)
#     surrogate accuracy:
    X_test = system.AI_model.load_data("../data/AIdata/daily_margin_test.csv")
    ens_test = system.AI_model.load_data("../data/AIdata/ens_test.csv")
    lol_test = system.AI_model.load_data("../data/AIdata/lol_test.csv")
    lol_hat, ens_hat = system.AI_model.predict(X_test)
    print(f"HGBRT RMSE (h/y): {mean_squared_error( lol_test, lol_hat, squared=False):.4f}")
    print(f"SVR RMSE (MWh/y): {mean_squared_error( ens_test, ens_hat, squared=False):.4f}")
    # Estimator:Full, Architecture:  Exact | HGB+Gre | HGB+SVR | Avg
    ml_hierarchy = ['OptimalNStore', 'AIGreedyNStore', 'AIModel', 'AvgStore']
    run_MLMC(system, samples, n_run, time_seconds, train_size, ml_hierarchy, use_joblib, verbose)

    # Estimator:Hybrid+base, Architecture:  Exact | HGB+Gre | Avg
    ml_hierarchy = ['OptimalNStore', 'AIGreedyNStore', 'AvgStore']
    run_MLMC(system, samples, n_run, time_seconds, train_size, ml_hierarchy, use_joblib, verbose)
    